In [1]:
import pandas as pd
import numpy as np

In [ ]:
tabla_historico = pd.read_excel('historica_Iberdrola_DIC_16-19.xls')
tabla_historico

In [ ]:
# Division of a global sequences into n_step size ones (X) with the prediction desired (y)
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Calculates a successfull ratio using less relative error than a maximum criterium
import keras.backend as K
def porcentaje_margen(y_true, y_pred):
    margen = 0.02
    yy = K.sum(K.cast(K.less(K.abs((y_pred/y_true)-1.0), margen), dtype=float))
    return yy/K.cast(K.shape(y_pred)[0], dtype=float)

In [ ]:
xmax = 15.0
T = 20
x_data, y_data = split_sequence(sequence=tabla_historico["Último"], n_steps=T)
x = x_data/xmax
y = y_data/xmax
n_features = 1 #Nº atributos

In [ ]:
x = x.reshape((x.shape[0], x.shape[1], n_features))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=2/3)

In [ ]:
model = Sequential()
model.add(SimpleRNN(10, return_sequences=True, input_shape=(T, n_features)))
model.add(SimpleRNN(5, return_sequences=False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])

In [ ]:
history=model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=200,verbose=1 ,steps_per_epoch=48, validation_steps=1)

In [ ]:
plt.plot(history.epoch, history.history['val_porcentaje_margen'], label='validation')
plt.plot(history.epoch, history.history['porcentaje_margen'], label='training')
plt.legend()
plt.title('Success Rate Evolution Simple RNN')
plt.xlabel('Epoch')
plt.show()

In [ ]:
model = Sequential()
model.add(LSTM(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])

In [ ]:
model = Sequential()
model.add(GRU(5, return_sequences=False, input_shape=(T, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=[porcentaje_margen])